In [ ]:
import sys
import biogeme.biogeme_logging as blog
import biogeme.exceptions as excep
import biogeme.biogeme as bio
import biogeme.distributions as dist
import biogeme.results as res
from biogeme import models
from biogeme.expressions import (
    Beta,
    RandomVariable,
    exp,
    log,
    Integrate,
)

from read_or_estimate import read_or_estimate

from optima import (
    read_data,
    male,
    age,
    haveChildren,
    highEducation,
       SocioProfCat,
    WaitingTimePT,
    Choice,
    TimePT_scaled,
    TimeCar_scaled,
    MarginalCostPT_scaled,
    CostCarCHF_scaled,
    distance_km_scaled,
    PurpHWH,
    PurpOther,
)

logger = blog.get_screen_logger(level=blog.INFO)
logger.info('Example m04_sequential_estimation.py')

In [ ]:
MODELNAME = 'm01_latent_variable'
try:
    struct_results = res.bioResults(pickle_file=f'{MODELNAME}.pickle')
except excep.BiogemeError:
    print(
        f'Run first the script {MODELNAME}.py in order to generate the '
        f'file {MODELNAME}.pickle, and move it to the directory saved_results'
    )
    sys.exit()
struct_betas = struct_results.get_beta_values()

In [ ]:
coef_intercept = struct_betas['coef_intercept']
coef_age_50_less = struct_betas['coef_age_50_less']
coef_male = struct_betas['coef_male']
coef_haveChildren = struct_betas['coef_haveChildren']
coef_highEducation = struct_betas['coef_highEducation']
coef_employees = struct_betas['coef_employees']


In [ ]:
omega = RandomVariable('omega')
density = dist.normalpdf(omega)
sigma_s = Beta('sigma_s', 1, None, None, 0)

In [ ]:
URB_PREF = (
    coef_intercept
        + coef_highEducation * highEducation
       + coef_employees * (SocioProfCat == 6)
    + coef_age_50_less * (age <= 50)
    + coef_male * male
    + coef_haveChildren * haveChildren
    + sigma_s * omega
)

In [ ]:
ASC_CAR = Beta('ASC_CAR', 0.94, None, None, 0)
ASC_PT = Beta('ASC_PT', 0, None, None, 1)
ASC_SM = Beta('ASC_SM', 0.35, None, None, 0)
BETA_COST_HWH = Beta('BETA_COST_HWH', -2.3, None, None, 0)
BETA_COST_OTHER = Beta('BETA_COST_OTHER', -1.9, None, None, 0)
BETA_DIST = Beta('BETA_DIST', -1.3, None, None, 0)
BETA_TIME_CAR_REF = Beta('BETA_TIME_CAR_REF', -6.1, None, 0, 0)
BETA_TIME_PT_REF = Beta('BETA_TIME_PT_REF', 0, None, 0, 0)
BETA_WAITING_TIME = Beta('BETA_WAITING_TIME', -0.075, None, None, 0)

In [ ]:
BETA_TIME_PT_UP = Beta('BETA_TIME_PT_UP', 1.5, None, None, 0)
BETA_TIME_PT = BETA_TIME_PT_REF + BETA_TIME_PT_UP * URB_PREF
BETA_TIME_CAR_UP = Beta('BETA_TIME_CAR_UP', -0.11, None, None, 0)
BETA_TIME_CAR = BETA_TIME_CAR_REF + BETA_TIME_CAR_UP * URB_PREF

In [ ]:
V0 = (
    ASC_PT
    + BETA_TIME_PT * TimePT_scaled
    + BETA_WAITING_TIME * WaitingTimePT
    + BETA_COST_HWH * MarginalCostPT_scaled * PurpHWH
    + BETA_COST_OTHER * MarginalCostPT_scaled * PurpOther
)

V1 = (
    ASC_CAR
    + BETA_TIME_CAR * TimeCar_scaled
    + BETA_COST_HWH * CostCarCHF_scaled * PurpHWH
    + BETA_COST_OTHER * CostCarCHF_scaled * PurpOther
)

V2 = ASC_SM + BETA_DIST * distance_km_scaled

In [ ]:
V = {0: V0, 1: V1, 2: V2}

In [ ]:
condprob = models.logit(V, None, Choice)

In [ ]:
loglike = log(Integrate(condprob * density, 'omega'))

In [ ]:
database = read_data()

In [ ]:
the_biogeme = bio.BIOGEME(database, loglike)
the_biogeme.modelName = 'm04_sequential_estimation'
the_biogeme.maxiter = 1000

In [ ]:
results = read_or_estimate(the_biogeme=the_biogeme, directory='saved_results')

In [ ]:
print(results.short_summary())

In [ ]:
printprint(results.short_summary())(f'Final log likelihood: {results.data.logLike:.3f}')
print(f'Output file: {results.data.htmlFileName}')

In [ ]:
results.get_estimated_parameters()

In [ ]:
print(results.short_summary())